# Imports and Overview

In [2]:
# Neural Network
import tensorflow as tf
from tensorflow.keras.models import Model
# Layer
from tensorflow.keras.layers import Dense, Layer, Conv2DTranspose, Conv2D, GlobalAveragePooling2D, Reshape, BatchNormalization, GRUCell, MaxPooling2D
import tensorflow_probability as tfp



# Buffer 
from tf_agents.replay_buffers import tf_uniform_replay_buffer

# Further support
import numpy as np
from tqdm.notebook import tqdm_notebook

# Environment
import gym
import highway_env
import random




# Experience Replay Buffer

In [3]:
# Has to save (Observation, action, reward, terminal state)
from numpy import float32


class Buffer:

    def __init__(
        self,
        batch_size=50,
        buffer_length=1000, 
        observation_size=(128,32),
        action_size=1
    ):
        '''
        Create replay buffer

        Buffer size = batch_size * buffer_length

        '''
        # Save batch size for other functions of buffer
        self.batch_size = batch_size

        # Tell buffer what data & which size to expect
        self.data_spec = (
            tf.TensorSpec(
                shape= observation_size,
                dtype=tf.dtypes.float64,
                name="Observation"
            ),
            tf.TensorSpec(
                shape=observation_size,
                dtype=tf.dtypes.float64,
                name="Next state"
            ),
            tf.TensorSpec(
                shape=[action_size],
                dtype=tf.dtypes.int32,
                name="Action"
            ),
            tf.TensorSpec(
                # Reward size
                shape=[1, 1],
                dtype=tf.dtypes.float64,
                name="Reward"
            ),
            tf.TensorSpec(
                shape=[1, 1],
                # Either 0 or 1 
                dtype=tf.dtypes.bool,
                name="Terminal State"
            )
        )

        # Create the buffer 
        self.buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
            self.data_spec, batch_size, buffer_length
        )

    def obtain_buffer_specs(self):
        return self.data_spec

    def add(self, items):
        '''
        length of items must be equal to batch size

        items: list or tuple of batched data from (50, 4)


        '''
        # Combine all values from "items" in tensor
        # Not sure wether we need tf.nest.map_structure
        self.batched_values = tf.nest.map_structure(
            lambda t: tf.stack([t] * self.batch_size),
            items
        )

        # Add to batch
        self.buffer.add_batch(self.batched_values)

    def sample(self):
        data = self.buffer.as_dataset(single_deterministic_pass=True).take(50)
        data = 




# Environment

In [4]:
class EnvironmentInteractor:

  def __init__(self, config, buffer, environment_name = "highway-fast-v0"):
    self.config = config

    self.env = gym.make(environment_name)    
    self.env.configure(config)

    self.buffer = buffer
    # Save sizes of the stupid tensors
    self.data_spec = self.buffer.obtain_buffer_specs()
  

  

  def create_trajectories(self, iterations):
    state = self.env.reset()
    



    for _ in range(iterations):
        action = self.env.action_space.sample()
        next_state, reward, done, _ = self.env.step(action)
        self.buffer.add((
            tf.constant(state, shape=self.data_spec[0].shape.as_list()),
            tf.constant(next_state, shape=self.data_spec[1].shape.as_list()),
            tf.constant(action, shape=self.data_spec[2].shape.as_list()),
            tf.constant(reward, shape=self.data_spec[3].shape.as_list()),
            tf.constant(done, shape=self.data_spec[4].shape.as_list())
        ))
        #print(state.shape)
        state = next_state
        
        if done:
          state = self.env.reset()


  def __del__(self):
    self.env.close()

c:\Users\User1\anaconda3\envs\iannwtf\lib\site-packages\highway_env\vehicle\objects.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.position = np.array(position, dtype=np.float)
c:\Users\User1\anaconda3\envs\iannwtf\lib\site-packages\highway_env\vehicle\controller.py:273: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in Num

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Iterator trajectories:
[(<tf.Tensor: shape=(128, 32), dtype=float64, numpy=
array([[100., 100., 100., ..., 100., 255., 100.],
       [100., 100., 100., ..., 100., 255., 100.],
       [100., 100., 100., ..., 100., 255., 100.],
       ...,
       [100., 100., 100., ..., 100., 255., 100.],
       [100., 100., 100., ..., 100., 255., 100.],
       [100., 100., 100., ..., 100., 255., 100.]])>, <tf.Tensor: shape=(128, 32), dtype=float64, numpy=
array([[100., 100., 100., ..., 100., 255., 100.],
       [100., 100., 100., ..., 100., 255., 100.],
       [100., 100., 100., ..., 100., 255., 100.],
       ...,
       [100., 100., 100., ..., 100., 255., 100.],
       [100., 100., 100., ..., 100., 255., 100.],
       [100., 100., 100., ..., 100., 255., 100.]])>, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1])>, <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[0.79799228]])>, <tf.Tensor: shape=(1, 1), 

# Parameters

In [6]:
# Image size
image_shape = (128,32)

# Long term memory of GRU
hidden_unit_size = 200

# Z in paper
stochastic_state_shape = (32,32)
stochastic_state_size = stochastic_state_shape[0] * stochastic_state_shape[1]

#
action_size = 1
#

#
mlp_hidden_layer_size = 100

# TODO different variable names for network inp/outp sizes




# World model

In [7]:
from importlib_metadata import distribution


class WorldModel:


    def __init__(self) -> None:
        super().__init__()


    def create_encoder(self, input_size=image_shape, output_size=hidden_unit_size):
        # Third dimension might be obsolete
        encoder_input = tf.keras.Input(shape=input_size)
        x = Conv2D(16, (3, 3), activation="elu", padding="same")(encoder_input) # 16 layers of filtered 192x48 features
        x = MaxPooling2D((2, 2), padding="same")(x) # 64 / 96x24
        x = Conv2D(32, (3, 3), activation="elu", padding="same")(x) # 64 / 96x24
        x = MaxPooling2D((2, 2), padding="same")(x) # 64 / 96x24
        x = Conv2D(64, (3, 3), activation="elu", padding="same")(x) # 64 / 48x12
        x = MaxPooling2D((2, 2), padding="same")(x) # 64 / 48x12
        x = GlobalAveragePooling2D()(x) # 64
        encoder_output = Dense(output_size, activation = "elu")(x)

        encoder = tf.keras.Model(encoder_input, encoder_output, name="Encoder")

        return encoder


    # Input size = 1024(z:32x32) + 200(size of hidden state)
    # Output size = game frame
    def create_decoder(
        self, 
        input_size=stochastic_state_size + hidden_unit_size, 
        output_size=image_shape
    ):
        # Third dimension might be obsolete
        decoder_input = tf.keras.Input(shape=input_size)
        # TODO WIE SCHLIMM IST EIN MLP HIER?
        x = Dense(256, activation= "elu")(decoder_input)
        x = Reshape((32, 8, 1))(x) 
        x = Conv2DTranspose(16, (3, 3), strides=2, activation="elu", padding="same")(x)
        x = BatchNormalization()(x)
        x = Conv2DTranspose(1, (3, 3), strides=2, activation="elu", padding="same")(x)
        # Might needs shape as Tensor
        decoder_output = tfp.layers.IndependentNormal(event_shape=output_size)(x)


        decoder = tf.keras.Model(decoder_input, decoder_output, name="Decoder")

        return decoder
    

        # Input: concatination of h and z
    # Output: float predicting the obtained reward
    def create_reward_predictor(
        self, 
        input_size=hidden_unit_size+stochastic_state_size,
        output_size=1
    ):
        reward_predictor_input = tf.keras.Input(shape=input_size)
        x = Dense(mlp_hidden_layer_size, activation="elu")(reward_predictor_input)
        x = Dense(mlp_hidden_layer_size, activation="elu")(x)
        x = Dense(mlp_hidden_layer_size)(x)
        # Creates indipendent normal distribution
        # Hope is that it learns to output variables over reward space [0,1]
        reward_predictor_output = tfp.layers.IndependentNormal()(x)

        create_reward_predictor = tf.keras.Model(
            reward_predictor_input,
            reward_predictor_output,
            name="create_reward_predictor"
        )

        return create_reward_predictor
    

        # Input: concatination of h and z
    # Output: float predicting the obtained reward
    def create_discount_predictor(
        self, 
        input_size=hidden_unit_size+stochastic_state_size,
        output_size=1
    ):
        discount_predictor_input = tf.keras.Input(shape=input_size)
        x = Dense(mlp_hidden_layer_size, activation="elu")(discount_predictor_input)
        x = Dense(mlp_hidden_layer_size, activation="elu")(x)
        # Create 1 output sampled from bernoulli distribution
        discount_predictor_output = tfp.layers.IndependentBernoulli()(x)

        create_discount_predictor = tf.keras.Model(
            discount_predictor_input,
            discount_predictor_output,
            name="create_discount_predictor"
        )

        return create_discount_predictor




class RSSM:


    def __init__(self) -> None:
        super().__init__()


    # 
    def create_stochastic_state_action_embedder(
        self, 
        input_size=stochastic_state_size + action_size,
        output_size=hidden_unit_size
    ):
        state_action_input = tf.keras.Input(shape=input_size)
        state_action_output = Dense(output_size, activation = "elu")(state_action_input)

        stochastic_state_action_embedder = tf.keras.Model(
            state_action_input,
            state_action_output,
            name="stochastic_state_action_embedder"
        )

        return stochastic_state_action_embedder


    # Contains GRU cell
    def create_rnn(
        self, 
        input_size=hidden_unit_size,
        output_size=hidden_unit_size
    ):
        rnn_input = tf.keras.Input(shape=input_size)
        rnn_output = GRUCell(output_size)(rnn_input)

        rnn = tf.keras.Model(
            rnn_input,
            rnn_output,
            name="rnn"
        )

        return rnn


    # Gets probabilities for each element of class in each category
    # Turns these (32x32) probabilities into categoricals (either 0 or 1)
    def sample_stochastic_state(self, logits):
        # logits Output from MLP
        #Onehot logits and create distribution from it (tfp.distrib)
        # sample from distribution = sample (32x32)
        # Apply softmax on sample to get probabillities = probs
        # Do sample = + probs - stop_grad(probs) (gradients)
        #return sample, gradients
        logits = tf.reshape(logits, shape=(*logits.shape[:-1], *stochastic_state_shape))
        logits_distribution = tfp.distributions.OneHotCategorical(logits)
        sample = logits_distribution.sample() 
        sample += logits_distribution.prob(sample) - tf.stop_gradient(logits_distribution.prob(sample))

        return sample


    # Z^ in paper
    def create_prior_stochastic_state_embedder(
        self, 
        input_size=hidden_unit_size,
        output_size=stochastic_state_size
    ):
        state_embedder_input = tf.keras.Input(shape=input_size)
        x = Dense(mlp_hidden_layer_size, activation="elu")(state_embedder_input)
        # Activation function removed
        state_embedder_output = Dense(output_size)(state_embedder_output)

        create_prior_stochastic_state_embedder = tf.keras.Model(
            state_embedder_input,
            state_embedder_output,
            name="create_prior_stochastic_state_embedder"
        )

        return create_prior_stochastic_state_embedder


    # Z in paper
    # Input size = concatenated output of RNN with output of CNN
    def create_posterior_stochastic_state_embedder(
        self, 
        input_size=hidden_unit_size+hidden_unit_size,
        output_size=stochastic_state_size
    ):
        state_embedder_input = tf.keras.Input(shape=input_size)
        x = Dense(mlp_hidden_layer_size, activation="elu")(state_embedder_input)
        # Activation function removed
        state_embedder_output = Dense(output_size)(state_embedder_output)

        create_posterior_stochastic_state_embedder = tf.keras.Model(
            state_embedder_input,
            state_embedder_output,
            name="create_posterior_stochastic_state_embedder"
        )

        return create_posterior_stochastic_state_embedder

# Test Everything

In [ ]:
buffer = Buffer(batch_size=1)
config = {
        "observation": {
            "type": "GrayscaleObservation",
            "observation_shape": (128, 32),
            "stack_size": 1,
            # weights for RGB conversion
            "weights": [0.01, 0.01, 0.98],  
            "scaling": 1.5,
        },
        # was at 2
        "policy_frequency": 1 
    }

environment_interactor = EnvironmentInteractor(config, buffer)
environment_interactor.create_trajectories(10)






dataset = buffer.sample()

iterator = iter(dataset)
print("Iterator trajectories:")
trajectories = []
for _ in range(3):
  t, _ = next(iterator)
  trajectories.append(t)

print(trajectories)

print(tf.nest.map_structure(lambda t: t.shape, trajectories))

# Actor Critic

# World model & agent training loops

# Hyperparam inits
Agent Data collection in environment + adding data to ERB (+ measure at which reward loop stops?) 
World model loop on data sampled from ERB
Agent training loop with world model feedback
 

# Function execution